In [1]:
#IMPORT MODULES
import numpy as np
import pandas as pd
import random
import math
import copy
import re
import sys
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
#IMPORT CSVs
players = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/playerlist.csv")
playergames = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/pca.csv")
games = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/gameresults.csv")
mpfile = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/mpfile.csv")

import pickle
with open("/Users/williamarnesen/Desktop/NBA_Model/nba_height_weight.p","rb") as f:
    heightweight = pickle.load(f)
    
heightweight["POR"][2021]['Carmelo Anthony'] = {'Height': 80, 'Weight': 240, 'Age': 'March 29, 1984'}
heightweight['NOP'][2021]['Brandon Ingram'] = {'Height': 79, 'Weight': 190, 'Age': 'September 2, 1997'}

with open("/Users/williamarnesen/Desktop/NBA_Model/nba_draft.p","rb") as f:
    draft = pickle.load(f)
    

FileNotFoundError: [Errno 2] File /Users/williamarnesen/Desktop/NBA_Model/pca.csv does not exist: '/Users/williamarnesen/Desktop/NBA_Model/pca.csv'

In [1502]:
#CONVERT CSVS TO EASIER FORMAT
names = list(playergames.values[:, 0])
player_game_values = playergames.values
gameresults = games.values
copied_gameresults = copy.deepcopy(gameresults)
mpresults = mpfile.values
playerdict = list(players.values[:,0])
gamescorespd = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/full_sheet_scaled.csv")
gamescores0 = gamescorespd.values
gamescores = copy.deepcopy(gamescores0)


In [1503]:
team_to_abbr_mapping = {
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Milwaukee Bucks": "MIL",
    "Atlanta Hawks": "ATL",
    "Phoenix Suns": "PHO",
    "Denver Nuggets": "DEN",
    "Utah Jazz": "UTA",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BRK",
    "Philadelphia 76ers": "PHI",
    "Houston Rockets": "HOU",
    "Miami Heat": "MIA",
    "Dallas Mavericks": "DAL",
    "Toronto Raptors": "TOR",
    "Portland Trail Blazers": "POR",
    "Indiana Pacers": "IND",
    "San Antonio Spurs": "SAS",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Cleveland Cavaliers": "CLE",
    "Sacramento Kings": "SAC",
    "Golden State Warriors": "GSW",
    "Washington Wizards": "WAS",
    "Charlotte Hornets": "CHO",
    "Memphis Grizzlies": "MEM",
    "New Orleans Pelicans": "NOP",
    "Detroit Pistons": "DET",
    "Minnesota Timberwolves": "MIN",
    "New York Knicks": "NYK",
    "Chicago Bulls": "CHI"
}

**MODEL FOR ENCODING BOX SCORES AS EMBEDDINGS (DECLARATION FOLLOWED BY TRAINING)

In [1504]:
##Train Embeddings
class model_v2(nn.Module):
    
    def __init__(self, dimensionality):
        super().__init__()
        
        self.__layer_one = nn.Linear(dimensionality, 4)
        self.__layer_two = nn.Linear(4, 1)
    
    def forward(self, performance):
        if (len(performance) == 1):
            performance.unsqueeze(0)
        return self.predict(self.encode(performance))
    
    def encode(self,performance):
        return self.__layer_one(performance)
    
    def predict(self, embedding):
        return self.__layer_two(torch.tanh(embedding))
        

In [1505]:
model2 = model_v2(20) #20 is the original
model2 = model2.double()
optimizer = optim.AdamW(model2.parameters(), lr=1e-5) # change the learning rate if you'd like
criterion = nn.MSELoss() # this doesn't include the regularization terms yet
keep = [2,3,5,6,8,9,11,12,14,15,16,17,18,20,33,34,35,42,43,44]

In [1506]:
#define validation
gamescorespd = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/full_sheet_scaled.csv")
gamescores0 = gamescorespd.values
gamescores = copy.deepcopy(gamescores0)

train=gamescores[0:len(gamescores)]
#test=gamescores[125000:len(gamescores)]

def test_on_validation2():
    counter=0
    total=0
    model2.eval()
    random.shuffle(test)
    for i in range(len(test)): #len(gamescores)
        if train[i][47]>10:
            gameval=test[i][keep].astype('double')
            correct_label=test[i][46]

            gamevalinput=torch.from_numpy(gameval)
            correct_label=torch.from_numpy(np.array(correct_label).astype('double'))

            with torch.no_grad():
                prediction = model2(gamevalinput.unsqueeze(0))
                loss = criterion(prediction, correct_label)
                counter+=1
                total += loss.item()
    print(total/counter)

In [1507]:
#THIS IS THE TRAINING
#gamescores[:,40] is the target vector
total=0
counter = 0
index = 0
batch_size = 1
train=gamescores

for epoch in range(10):
    train = random.sample(list(train), k=len(train))

    for i in range(len(train)): #len(gamescores)
        if train[i][47]>10:
            gameval=train[i][keep].astype('double') #re-add 20
            correct_label=train[i][46]
            gamevalinput=torch.from_numpy(gameval)
            correct_label=torch.from_numpy(np.array(correct_label).astype('double'))

            with torch.enable_grad():
                prediction = model2(gamevalinput.unsqueeze(0))
                loss = criterion(prediction, correct_label)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                counter+=1
                index += batch_size                                    
                total += loss.item()
                if counter % 10000 == 0:
                    print(counter,total/counter)
    print(epoch,total/counter)

10000 0.6050105718777063
20000 0.5499954382933412
30000 0.5013270981524695
40000 0.4658489552085388
50000 0.435307172620822
60000 0.4099663356600888
70000 0.38956567594942954
80000 0.37245683212774044
90000 0.35792432619608555
100000 0.3455773652579795
110000 0.33598402563190877
120000 0.3267717728344689
130000 0.3187741984175156
140000 0.311842176810749
0 0.3106793553396095
150000 0.30588487347640536
160000 0.3000034941031079
170000 0.29573704499391895
180000 0.2915173983051869
190000 0.2879667124919457
200000 0.28447369268078165
210000 0.28127150774040904
220000 0.278485459345781
230000 0.27573390536621206
240000 0.2736143615098985
250000 0.27136861635769177
260000 0.26921452694643866
270000 0.26702400341152704
280000 0.2652188964973518
1 0.264573552034771
290000 0.2635685378924142
300000 0.262159820684741
310000 0.26083857737936006
320000 0.25944125843428745
330000 0.25815185073435787
340000 0.25700088246086156
350000 0.25601679462932886
360000 0.2548354772571441
370000 0.2540247213

**CALCULATES PLAYER'S EMBEDDINGS

In [1534]:
gamescorespd = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/full_sheet_scaled.csv")
gamescores0 = gamescorespd.values
gamescores = copy.deepcopy(gamescores0)

In [1539]:
#THIS GETS EACH PLAYER'S EMBEDDINGS AT EACH POINT IN TIME (JUMP HERE)
def get_avg_player():
    tot_vals=torch.zeros(4)
    counter = 0
    for player in embeddings:
        tot_vals+=embeddings[player]['current']
        counter += 1
    avg_player = tot_vals/counter
    return(avg_player)


embeddings={}
gp={}

for i in range(len(gamescores)):
    player=gamescores[i,0]
    embeddings[player]={'current': torch.zeros(4)}
    gp[player]=0

for i in range(len(gamescores)):
    if gamescores[i,47]>2:
        game=gamescores[i,40]
        if i==0:
            old_game=gamescores[i,40]
        if game != old_game:
            if game == gameresults[1311,0]:
                for player in embeddings:
                    embeddings[player]['2015']=embeddings[player]['current']
            if game == gameresults[2627,0]:
                for player in embeddings:
                    embeddings[player]['2016']=embeddings[player]['current']
            if game == gameresults[3936,0]:
                for player in embeddings:
                    embeddings[player]['2017']=embeddings[player]['current']
            if game == gameresults[5248,0]:
                for player in embeddings:
                    embeddings[player]['2018']=embeddings[player]['current']
            if game == gameresults[6560,0]:
                for player in embeddings:
                    embeddings[player]['2019']=embeddings[player]['current']
            if game == gameresults[7701,0]:
                for player in embeddings:
                    embeddings[player]['2020']=embeddings[player]['current']
        
        player=gamescores[i,0]
        gameval=gamescores[i,keep].astype('double')
        gamevalinput=torch.from_numpy(gameval)
        new_val=model2.encode(gamevalinput)

        prev_gp=gp[player]
        new_gp=prev_gp+1
        gp[player]=new_gp

        existing=embeddings[player]['current']
        new_embedding=existing+(new_val-existing)/(3*math.sqrt(new_gp)) #*(48/mp[player][game])

        embeddings[player][game]=new_embedding
        embeddings[player]['current']=new_embedding
        old_game = game

**Rookie Adjustments**

In [1471]:
class model_rookie_projections(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.__mean_center = nn.Parameter(torch.zeros(4))
        self.__layer_one = nn.Linear(4, 4) #input is height, weight, age, draft position
        #self.__layer_two = nn.Linear(2, 4) 
    
        #self.dropout = nn.Dropout(1/7)

    def forward(self, nextyear):
        #return self.__layer_two(F.gelu(self.__layer_one(nextyear - self.__mean_center)))
        return self.__layer_one(nextyear - self.__mean_center)
    
    def score(self, nextyear):
        print(nextyear - self.__mean_center)
        print(self.__layer_one(nextyear - self.__mean_center))
        print(F.gelu(self.__layer_one(nextyear - self.__mean_center)))
        print(self.__layer_two(F.gelu(self.__layer_one(nextyear - self.__mean_center))))
        print(self(nextyear))
        return self(nextyear)

In [1472]:
model_rookies = model_rookie_projections()
model_rookies = model_rookies.double()
optimizer_rookies = optim.AdamW(model_rookies.parameters(), lr=1e-5) # change the learning rate if you'd like
criterion = nn.MSELoss() # this doesn't include the regularization terms yet

In [1473]:
#train the rookie model
draft_names = [player for player in filter(lambda name: len(name) > 2, draft)]

for epoch in range(501):
    counter = 0
    total = 0
    random.shuffle(draft_names)
    for player in draft_names:
        year = draft[player][0]
        if year>=2015 and year<2020:
            if player in embeddings:
                nextyear = str(year + 1)
                nextyearint = year + 1
                correct_label = embeddings[player][nextyear]
                for team in NBA:
                    roster = heightweight[team][nextyearint]
                    if player in roster:
                        if 'height' in roster[player]:
                                height = int(roster[player]['height'])
                                weight = int(roster[player]['weight'])
                                dob = roster[player]['age']
                        else:
                            height = int(roster[player]['Height'])
                            weight = int(roster[player]['Weight'])
                            dob = roster[player]['Age']
                        yearextr=re.search('\d{4}',dob)
                        yob = int(yearextr.group(0))
                        age = year - yob

                        if player in draft:
                            draft_pos=draft[player][1]
                        else:
                            draft_pos=61

                        input_vec = torch.tensor(np.asarray([height, weight, age, draft_pos]),dtype=torch.double)
                        input_vec = input_vec.detach()

                        with torch.enable_grad():
                            correct_label = embeddings[player][nextyear].detach() 
                            prediction = model_rookies(input_vec.unsqueeze(0))
                            loss = criterion(prediction, correct_label)
                            loss.backward()
                            optimizer_rookies.step()
                            optimizer_rookies.zero_grad()
                            total += loss.item()
                            counter+=1
                            index += batch_size                                    
                            total += loss.item()

    if epoch % 10 == 0:
        print(epoch,total/counter)

0 10841.148111335175
10 9023.372185368507
20 7469.047063258098
30 6121.518672011646
40 4957.400129707604
50 3966.7075006924824
60 3136.3264766788166
70 2442.498923426049
80 1865.57386403533
90 1397.4313307092973
100 1023.9993847116988
110 724.7826802258356
120 487.4917947500877
130 308.22474494534873
140 183.98095870098894
150 109.2920960490595
160 73.16128673063336
170 58.876725559996785
180 53.203717442840855
190 50.036928067307166
200 47.52606569316066
210 45.247753945025416
220 43.10469361567712
230 41.04547726850437
240 39.08894859979362
250 37.20027352100996
260 35.39396613893962
270 33.68343142492297
280 32.01972107313632
290 30.428646608846304
300 28.916555202585293
310 27.44776677119377
320 26.05580932551347
330 24.716701156200376
340 23.412022566223744
350 22.17082077256414
360 20.968125530357746
370 19.822986010849636
380 18.70825998324559
390 17.64644611436781
400 16.615401876709768
410 15.628497690688546
420 14.681097129258315
430 13.767714296585064
440 12.901072967415233


In [1510]:
def construct_description_vector(playername, year):
    nextyearint = year + 1
    nextyear = str(nextyearint)
    for team in NBA:
        roster = heightweight[team][nextyearint]
        if player in roster:
            if 'height' in roster[player]:
                    height = int(roster[player]['height'])
                    weight = int(roster[player]['weight'])
                    dob = roster[player]['age']
            else:
                height = int(roster[player]['Height'])
                weight = int(roster[player]['Weight'])
                dob = roster[player]['Age']
            yearextr=re.search('\d{4}',dob)
            yob = int(yearextr.group(0))
            age = year - yob

            if player in draft:
                draft_pos=draft[player][1]
            else:
                draft_pos=61

            return torch.tensor(np.asarray([height, weight, age, draft_pos]), dtype=torch.double).detach()

In [1475]:
#input the rookie ratings for 2020
result_list = []
for player in draft:
    year = draft[player][0]
    if year == 2020 and len(player)>0 and player in embeddings:
        input_vec = construct_description_vector(player, year)
        if input_vec != None:
            with torch.no_grad():
                proj = model_rookies(input_vec.unsqueeze(0))
            with torch.no_grad():
                pred = model2.predict(proj)
            #print(player)
            #print("\t",proj[0].numpy())
            #print("\t", pred[0].item())
            result_list.append((player, pred[0].item()))
        
        """
        if player in embeddings:
            embeddings[player]['2020'] = proj.squeeze()
        else:
            embeddings[player]={'2020': proj, 'current': proj}
            embeddings[player] = embeddings[player].squeeze()
        """

sorted_list = sorted(result_list, key=lambda pair: pair[1], reverse=True)
for i, (player, score) in enumerate(sorted_list):
    print(i + 1, player, score)

1 Nick Richards 0.6464780639627723
2 Sam Merrill 0.6166567725779275
3 Robert Woodard 0.5888973189880811
4 Reggie Perry 0.5196989023058356
5 Jordan Nwora 0.49024461610645437
6 Elijah Hughes 0.47135949770315694
7 Desmond Bane 0.2940061031505046
8 Daniel Oturu 0.15505159826274673
9 Udoka Azubuike 0.13285783053807154
10 Malachi Flynn 0.09948897325843531
11 Payton Pritchard 0.0256420671443765
12 CJ Elleby -0.06184916657412698
13 Jahmi'us Ramsey -0.06565298973981398
14 Isaiah Stewart -0.1472626472227448
15 Tre Jones -0.23293544208483707
16 Obi Toppin -0.25501597947944293
17 Théo Maledon -0.36338101505469256
18 Immanuel Quickley -0.37013099040415626
19 Isaiah Joe -0.4102926018554791
20 Jaden McDaniels -0.4471576041399605
21 Aaron Nesmith -0.4594137619128625
22 Anthony Edwards -0.47821405608996653
23 Tyrese Maxey -0.5257509609068478
24 Tyrell Terry -0.5434889184284268
25 Precious Achiuwa -0.5666112780719225
26 Isaac Okoro -0.6089550523716996
27 R.J. Hampton -0.6189404130159313
28 Saddiq Bey -0

**GETS TEAM AFFILIATIONS

In [1511]:
East=['BRK','PHI','CHI','NYK','ATL','DET','CLE','ORL','TOR','MIA','CHO','WAS','BOS','IND','MIL']
West=['LAL','UTA','MEM','NOP','GSW','HOU','DAL','DEN','SAC','OKC','POR','MIN','PHO','LAC','SAS']
NBA=['BRK','PHI','CHI','NYK','ATL','DET','CLE','ORL','TOR','MIA','CHO','WAS','BOS','IND','MIL',
    'LAL','UTA','MEM','NOP','GSW','HOU','DAL','DEN','SAC','OKC','POR','MIN','PHO','LAC','SAS']

In [1512]:
#INITIALIZE DICTIONARIES FOR MP, TEAM SCORES, AND PLAYER AFFILIATIONS
affiliations={}
mp={}
abilities={}

embeddings['Michael Porter Jr.']=embeddings['Michael Porter']
embeddings['Kevin Porter Jr.']=embeddings['Kevin Porter']
embeddings['Larry Nance Jr.']=embeddings['Larry Nance']
embeddings['Tim Hardaway Jr.']=embeddings['Tim Hardaway']
embeddings['Gary Trent Jr.']=embeddings['Gary Trent']
embeddings['Marvin Bagley III']=embeddings['Marvin Bagley']
embeddings['Jaren Jackson Jr.']=embeddings['Jaren Jackson']
embeddings['Kelly Oubre Jr.']=embeddings['Kelly Oubre']


for i in range(len(gamescores)):
    player = gamescores[i,0]
    affiliations[player]={'current':None}
    mp[player]={'current':None}
    
for i in range(len(gamescores)):
    game = gamescores[i,40]
    player=gamescores[i,0]
    team=gamescores[i,37]
    affiliations[player][game]=team
    if gamescores[i,47]>2:
        mp[player][game]=gamescores[i,47]
    abilities[game]={}
    

affiliations['Michael Porter Jr.']=affiliations['Michael Porter']
affiliations['Kevin Porter Jr.']=affiliations['Kevin Porter']
affiliations['Larry Nance Jr.']=affiliations['Larry Nance']
affiliations['Tim Hardaway Jr.']=affiliations['Tim Hardaway']
affiliations['Gary Trent Jr.']=affiliations['Gary Trent']
affiliations['Marvin Bagley III']=affiliations['Marvin Bagley']
affiliations['Jaren Jackson Jr.']=affiliations['Jaren Jackson']
affiliations['Kelly Oubre Jr.']=affiliations['Kelly Oubre']

#people out all year or have inconsistent names
mp['Klay Thompson']['projected']=0
mp['Jonathan Isaac']['projected']=0
mp['Michael Porter Jr.']=mp['Michael Porter']
mp['Kevin Porter Jr.']=mp['Kevin Porter']
mp['Larry Nance Jr.']=mp['Larry Nance']
mp['Tim Hardaway Jr.']=mp['Tim Hardaway']
mp['Gary Trent Jr.']=mp['Gary Trent']
mp['Marvin Bagley III']=mp['Marvin Bagley']
mp['Jaren Jackson Jr.']=mp['Jaren Jackson']
mp['Kelly Oubre Jr.']=mp['Kelly Oubre']

for game in gameresults:
    gamename = game[0]
    winner = game[1]
    loser = game[2]
    winnerval = np.zeros(4)
    loserval = np.zeros(4)
    for player in embeddings:
        if gamename in embeddings[player]:
            if affiliations[player][gamename] == winner:
                with torch.no_grad():
                    winnerval = winnerval + embeddings[player][gamename].numpy() * (mp[player][gamename]/(48*5))
            else:
                with torch.no_grad():
                    loserval = loserval + embeddings[player][gamename].numpy() * (mp[player][gamename]/(48*5))
    abilities[gamename]['winner']=winnerval
    abilities[gamename]['loser']=loserval

In [1513]:
#GET MINUTES
mp538 = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/mp538.csv")
mpval = mp538.values

#people out all year or have inconsistent names
mp['Klay Thompson']['projected']=0
mp['Jonathan Isaac']['projected']=0
mp['Michael Porter Jr.']=mp['Michael Porter']
mp['Kevin Porter Jr.']=mp['Kevin Porter']
mp['Larry Nance Jr.']=mp['Larry Nance']
mp['Tim Hardaway Jr.']=mp['Tim Hardaway']
mp['Gary Trent Jr.']=mp['Gary Trent']
mp['Marvin Bagley III']=mp['Marvin Bagley']
mp['Jaren Jackson Jr.']=mp['Jaren Jackson']
mp['Kelly Oubre Jr.']=mp['Kelly Oubre']


for player in mp:
    mp[player]['projected']=0
for i in range(len(mpval)):
    player=mpval[i,0]
    if player in mp:
        mp[player]['projected']=mpval[i,1]
    mp['Klay Thompson']['projected']=0
    mp['Jonathan Isaac']['projected']=0
    mp['Michael Porter Jr.']=mp['Michael Porter']
    mp['Kevin Porter Jr.']=mp['Kevin Porter']
    mp['Larry Nance Jr.']=mp['Larry Nance']
    mp['Tim Hardaway Jr.']=mp['Tim Hardaway']
    mp['Gary Trent Jr.']=mp['Gary Trent']
    mp['Marvin Bagley III']=mp['Marvin Bagley']
    mp['Jaren Jackson Jr.']=mp['Jaren Jackson']
    mp['Kelly Oubre Jr.']=mp['Kelly Oubre']

for team in NBA:
    tot_min = 0
    roster = heightweight[team][2021]
    for player in roster:
        if player in mp:
            tot_min += mp[player]['projected']
    for player in roster:
        if player in mp:
            mp[player]['projected']/=(tot_min/240)

**PREDICTING CHANGES IN EMBEDDINGS FROM YEAR TO YEAR

In [1514]:
#PREDICT YEAR-TO-YEAR IMPROVEMENTS
#now trying to predict scores embeddings
class model_v1(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.__layer_one = nn.Linear(8, 4)
        self.__layer_two = nn.Linear(4, 4)
    
        #self.dropout = nn.Dropout(1/7)

    def forward(self, nextyear):
        
        nextyear_score = self.__layer_two(F.relu(self.__layer_one(nextyear)))
        return nextyear_score


In [1515]:
model_players = model_v1()
model_players = model_players.double()
optimizer_players = optim.AdamW(model_players.parameters(), lr=2e-4) # change the learning rate if you'd like
criterion = nn.MSELoss() # this doesn't include the regularization terms yet

In [1516]:
#do the training for year-to-year improvements
counter = 0
total = 0
yearlist=['2015','2016','2017','2018','2019']
for epoch in range(150):
    for yearstr in yearlist:
        year=int(yearstr)
        nextyear=int(yearstr)+1
        nextyearstr=str(nextyear)
        for player in embeddings:
            for team in heightweight:
                if player in heightweight[team][year]:
                    if player not in mp:
                        continue
                    skills = embeddings[player][yearstr]
                    nextskills=embeddings[player][nextyearstr]
                    if torch.max(skills) == 0 and torch.min(skills) == 0:
                        continue
                    if type(skills)!=int:
                        if skills[0] != nextskills[0] or skills[1] != nextskills[1] or skills[0] != 0 or skills[1] != 0:
                            if 'height' in heightweight[team][year][player]:
                                height = int(heightweight[team][year][player]['height'])
                                weight = int(heightweight[team][year][player]['weight'])
                                dob = heightweight[team][year][player]['age']
                            else:
                                height = int(heightweight[team][year][player]['Height'])
                                weight = int(heightweight[team][year][player]['Weight'])
                                dob = heightweight[team][year][player]['Age']
                            if player in draft:
                                draft_pos=draft[player][1]
                            else:
                                draft_pos=61
                            yearextr=re.search('\d{4}',dob)
                            yob = int(yearextr.group(0))
                            age = year - yob


                            descriptors = torch.tensor(np.asarray([height, weight, age, draft_pos]))
                            input_vec = torch.cat([skills, descriptors], dim=0)
                            input_vec = input_vec.detach()



                            with torch.enable_grad():
                                correct_label = nextskills.detach() - skills.detach()                                
                                prediction = model_players(input_vec.unsqueeze(0))
                                loss = criterion(prediction, correct_label)
                                loss.backward()
                                optimizer_players.step()
                                optimizer_players.zero_grad()
                                total += loss.item()
                                counter+=1
                                index += batch_size                                    
                                total += loss.item()
    print(epoch,total/counter)
                               

                        

0 7.622337063761295
1 3.8417641924190047
2 2.566640329899015
3 1.9285286199042884
4 1.5456601822033231
5 1.2904145829904679
6 1.1080962995006296
7 0.9713575869665784
8 0.865005254999879
9 0.7799233894267326
10 0.71031095395781
11 0.6523005910670446
12 0.6032148993902428
13 0.5611414493815554
14 0.5246777927073597
15 0.4927720931174385
16 0.4646200052439786
17 0.43959592713423645
18 0.41720596250973035
19 0.39705499434767483
20 0.37882316601057703
21 0.3622487766132153
22 0.34711563846779814
23 0.3332435951678324
24 0.3204813153318639
25 0.3087007493294315
26 0.2977928178456977
27 0.2876640243250878
28 0.27823376828865787
29 0.26943219598798995
30 0.2611984670615587
31 0.25347934619302936
32 0.24622805083168361
33 0.23940330225629938
34 0.23296853931379427
35 0.22689126320142833
36 0.2211424885005416
37 0.21569628088917525
38 0.2105293659758277
39 0.20562079680814752
40 0.20095167003889075
41 0.1965048826395986
42 0.19226492256120375
43 0.18821768794091776
44 0.1843503304148667
45 0.180

In [1549]:
for player in embeddings:
    for team in heightweight:
        if player in heightweight[team][2021]:
            
            if mp[player]["projected"] == 0 or player not in embeddings:
                continue
            skills = embeddings[player]['2020']
            if torch.max(skills) == 0 and torch.min(skills) == 0:
                continue
            if type(skills)!=int:
                if 'height' in heightweight[team][2021][player]:
                    height = int(heightweight[team][2021][player]['height'])
                    weight = int(heightweight[team][2021][player]['weight'])
                    dob = heightweight[team][2021][player]['age']
                else:
                    height = int(heightweight[team][2021][player]['Height'])
                    weight = int(heightweight[team][2021][player]['Weight'])
                    dob = heightweight[team][2021][player]['Age']
                yearextr=re.search('\d{4}',dob)
                yob = int(yearextr.group(0))
                age = year - yob
                
                if player in draft:
                    draft_pos=draft[player][1]
                else:
                    draft_pos=61


                descriptors = torch.tensor(np.asarray([height, weight, age, draft_pos]))
                input_vec = torch.cat([skills, descriptors], dim=0)
                input_vec = input_vec.detach()
                
                prediction = model_players(input_vec.unsqueeze(0))
                embeddings[player]['predicted'] = prediction+embeddings[player]['2020']
                    

**CREATING DATA, DEFININING, AND TRAINING MODEL FOR CONVERTING PLAYER EMBEDDINGS TO TEAM EMBEDDINGS TO WINS

In [1518]:
def create_team_list(team, game_id):
    team_list = []
    for player in filter(lambda name: game_id in embeddings[name], embeddings):
        if affiliations[player][game_id] == team:
            minutes_tensor = torch.tensor(mp[player][game_id], dtype=torch.float32)
            embedding_plus_minutes = torch.cat([embeddings[player][game_id], minutes_tensor.unsqueeze(0)], dim=0)
            team_list.append(embedding_plus_minutes.float().detach())
    return team_list

pc_abilities = {}
for game in trainset:
    game_id, winner, loser = game
    pc_abilities[game_id] = {}
    pc_abilities[game_id]["winner"] = create_team_list(winner, game_id)
    pc_abilities[game_id]["loser"] = create_team_list(loser, game_id)

In [1519]:
class PlayerCombinerModel(nn.Module):
        
    def __init__(self):
        super().__init__()
        
        EMBEDDING_SIZE = 4
        self.__player_layer = nn.Linear(EMBEDDING_SIZE + 1, 1)
        self.__team_layer_one = nn.Linear(EMBEDDING_SIZE * 2, 4)
        self.__team_layer_two = nn.Linear(4, 1)
    
    def forward(self, team_one_player_lists, team_two_player_lists):
        
        def score(combined):
            return self.__team_layer_two(torch.tanh(self.__team_layer_one(combined)))
        
        outputs = []
        for team_one_player_list, team_two_player_list in zip(team_one_player_lists, team_two_player_lists):
            team_one_embedding = self.find_team_embedding(team_one_player_list)
            team_two_embedding = self.find_team_embedding(team_two_player_list)
            
            output_one = score(torch.cat([team_one_embedding, team_two_embedding], dim=0))
            output_two = score(torch.cat([team_two_embedding, team_one_embedding], dim=0))
            outputs.append(torch.softmax(torch.cat([output_one, output_two], dim=0), dim=0))
            
        return torch.stack(outputs)
    
    def find_team_embedding(self, tensor_list):
        stacked_list = torch.stack(tensor_list)
        weights = self.__player_layer(stacked_list)
        normalized_weights = F.softmax(weights, dim=0)
        return torch.sum((normalized_weights * stacked_list[:, 0:4]), dim=0).squeeze()

In [1524]:
pc_model = PlayerCombinerModel()
pc_optimizer = optim.AdamW(pc_model.parameters(), lr=5e-4) # change the learning rate if you'd like

In [1525]:
counter = 0
batch_size = 32
pc_model.train()
for epoch in range(25):
    total = 0
    average_prediction = 0
    trainset = list(copy.deepcopy(gameresults))
    trainset = random.sample(trainset, len(trainset))
    index = 0
    counter = 0
    while index < len(trainset):
        batch_winners = []
        batch_losers = []
        for game in trainset[index:min(index+batch_size, len(trainset))]:
            gamename = game[0]
            winnerval = pc_abilities[gamename]['winner']
            loserval = pc_abilities[gamename]['loser']
            if len(winnerval) > 0 and len(loserval) > 0:
                batch_winners.append(winnerval)
                batch_losers.append(loserval)
        with torch.enable_grad():
            prediction = pc_model(batch_winners, batch_losers)
            loss = criterion(prediction[:, 0], torch.ones([len(batch_winners)]))
            loss.backward()
            pc_optimizer.step()
            pc_optimizer.zero_grad()
            total += loss.item()
            counter += 1
            average_prediction += torch.sum(prediction[:, 0]).item()
        index += batch_size
    
    if epoch % 1 == 0:
        print(counter, total / counter, average_prediction / len(trainset))

244 0.24953246806732943 0.5001893758160902
244 0.23201485668293764 0.5203660901836994
244 0.2132958324595553 0.5504603758569242
244 0.20511019455849147 0.5723722972109876
244 0.20292580872774124 0.5841946347513665
244 0.2013928721674153 0.5897495856015419
244 0.2007740640127268 0.5931921021184455
244 0.1998450127659274 0.5952381049759283
244 0.1995269146244057 0.5959187581177542
244 0.19959425639177933 0.5973852738316691
244 0.19938508744855396 0.5982399580104798
244 0.19914659564612341 0.597586680347325
244 0.19868045706362997 0.6005784670920482
244 0.19833617749028518 0.6002567335388654
244 0.19824993995125176 0.6000071041994659
244 0.19857810672800072 0.6011170390332573
244 0.19805343073533208 0.6007323105414912
244 0.19879556233521367 0.6013719639925234
244 0.19831590201766763 0.601882351609306
244 0.19803548549286654 0.6018147518211894
244 0.1980746436375575 0.6034774623983623
244 0.19798589830637955 0.6020584414734632
244 0.1975154596396157 0.6027179817920479
244 0.19840820506215

In [1552]:
projected_abilities = {}
team_abilities = {}
for team in NBA:
    team_list = []
    for player in filter(lambda name: name in heightweight[team][2021], embeddings):
        minutes_tensor = torch.tensor(mp[player]['projected'], dtype=torch.float32)
        predicted_score = embeddings[player]['current'].squeeze() if 'current' in embeddings[player] else embeddings[player]['current']
        embedding_plus_minutes = torch.cat([predicted_score, minutes_tensor.unsqueeze(0)], dim=0)
        team_list.append(embedding_plus_minutes.float().detach())
    projected_abilities[team] = team_list
    team_abilities[team] = pc_model.find_team_embedding(team_list)

**FINDING EXPECTED NUMBER OF WINS

In [1553]:
#GENERATE POTENTIAL MATCHUPS
pot_matchups = {}
counter = 0
iterator = 0
incount={}
for team in NBA:
    incount[team]=iterator
    iterator+=1
for team in NBA:
    for opponent in NBA:
        if incount[team]<incount[opponent]:
            idval=team+opponent
            if team in East and opponent in East:
                maxval = 3
            if team in West and opponent in West:
                maxval = 3
            if team in East and opponent in West:
                maxval = 2 #there cannot be team in West and opp in East because the East teams go first 
            pot_matchups[idval]={'played': 0, 'max': maxval}
            counter+=1
    
#GET REMAINING MATCHUPS
curr_season = gameresults[7702:len(gameresults),:]

for i in range(len(curr_season)):
    team1=team_to_abbr_mapping[curr_season[i,1]]
    team2=team_to_abbr_mapping[curr_season[i,2]]
    
    if incount[team1]<incount[team2]:
        idval=team1+team2
    else:
        idval=team2+team1
    pot_matchups[idval]['played']+=1

record={}
for team in NBA:
    record[team]={'W': 0, "L": 0}
    
#get current wins
for i in range(len(curr_season)):
    winner_abbr=team_to_abbr_mapping[curr_season[i,1]]
    record[winner_abbr]['W']+=1
    
    loser_abbr=team_to_abbr_mapping[curr_season[i,2]]
    record[loser_abbr]['L']+=1


In [1554]:
for gamename in pot_matchups:
    diff = pot_matchups[gamename]['max']-pot_matchups[gamename]['played']
    team = re.search('^.{3}',gamename).group(0)
    opponent = re.search('.{3}$',gamename).group(0)
    for rep in range(diff):
        proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
        record[team]['W']+=proj[0,0]
        record[team]['L']+=proj[0,1]
        record[opponent]['W']+=proj[0,1]
        record[opponent]['L']+=proj[0,0]



In [1555]:
sorted_team_list = sorted([(team, record[team]["W"].item()) for team in record], key=lambda x: x[1], reverse=True)
for i, (team, wins) in enumerate(sorted_team_list):
    print(i + 1, team, round(wins,2))

1 LAC 47.35
2 LAL 46.98
3 MIL 44.05
4 DEN 43.56
5 PHI 42.93
6 PHO 41.94
7 BOS 40.84
8 POR 40.6
9 MIA 40.26
10 UTA 39.9
11 HOU 39.64
12 TOR 38.84
13 BRK 38.28
14 IND 37.49
15 ATL 37.43
16 DAL 36.78
17 SAS 36.43
18 NOP 34.35
19 SAC 33.65
20 WAS 33.53
21 GSW 32.91
22 MEM 31.9
23 ORL 30.4
24 OKC 30.0
25 MIN 29.38
26 CLE 27.75
27 DET 27.54
28 CHO 26.94
29 NYK 26.1
30 CHI 22.26


In [1575]:
#FROM INITIAL
projected_abilities = {}
team_abilities = {}
for team in NBA:
    team_list = []
    for player in filter(lambda name: name in heightweight[team][2021], embeddings):
        minutes_tensor = torch.tensor(mp[player]['projected'], dtype=torch.float32)
        predicted_score = embeddings[player]['projected'].squeeze() if 'projected' in embeddings[player] else embeddings[player]['current']
        embedding_plus_minutes = torch.cat([predicted_score, minutes_tensor.unsqueeze(0)], dim=0)
        team_list.append(embedding_plus_minutes.float().detach())
    projected_abilities[team] = team_list
    team_abilities[team] = pc_model.find_team_embedding(team_list)
    
wins={}
for team in NBA:
    wins[team]=0
for team in East:
    for epoch in range(3):
        for opponent in East:
            if team!=opponent:
                proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
                wins[team]+=proj[0,0]
    for epoch in range(2):
        for opponent in West:
            if team!=opponent:
                proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
                wins[team]+=proj[0,0]
for team in West:
    for epoch in range(3):
        for opponent in West:
            if team!=opponent:
                proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
                wins[team]+=proj[0,0]
    for epoch in range(2):
        for opponent in East:
            if team!=opponent:
                proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
                wins[team]+=proj[0,0]   
                
sorted_teams = sorted([(team, wins[team].item()) for team in wins], key=lambda x: wins[x[0]].item(), reverse=True)
for i, (team, wins) in enumerate(sorted_teams):
    print(i + 1, team, round(wins,2))

1 LAC 46.96
2 LAL 46.25
3 DEN 45.54
4 MIL 44.88
5 PHI 42.15
6 MIA 41.4
7 BOS 41.24
8 PHO 40.22
9 HOU 40.17
10 TOR 40.12
11 POR 40.01
12 UTA 39.54
13 BRK 38.6
14 DAL 37.07
15 SAS 36.74
16 IND 36.33
17 ATL 36.2
18 WAS 36.14
19 NOP 33.58
20 GSW 33.36
21 SAC 32.49
22 MEM 32.19
23 OKC 30.97
24 MIN 29.22
25 DET 28.7
26 ORL 28.27
27 CHO 26.97
28 CLE 26.9
29 NYK 25.78
30 CHI 22.02


**FINDING BEST PLAYERS CURRENTLY

In [1550]:
running = torch.zeros(4, dtype=torch.float32)
for team in NBA:
    running += team_abilities[team]
avg = running/30

minutes_tensor = torch.tensor(48, dtype=torch.float32).unsqueeze(0).float()
base_list = [torch.cat([avg, minutes_tensor], dim=0).float() for i in range(5)]

mvp={}
for team in NBA:
    roster = heightweight[team][2021]
    for player in roster:
        if player in embeddings:
            if 'predicted' in embeddings[player]:
                new_list = [element for element in base_list]
                minutes_tensor = torch.tensor(48, dtype=torch.float32).unsqueeze(0).float()
                new_list[0] = torch.cat([embeddings[player]['current'].squeeze().float(), minutes_tensor], dim=0)
                with torch.no_grad():
                    #proj=pc_model([new_list],[base_list])
                    proj = model2.predict(embeddings[player]['predicted'].double())
                mvp[player]=proj[0]
                counter=0

sorted_list = sorted([(player, mvp[player].item()) for player in filter(lambda x: len(embeddings[x]) > 75,  mvp)], key=lambda x: mvp[x[0]].item(), reverse=True)

for i, (player, score) in enumerate(sorted_list):
    print(i+1,"\t", player, "\t", round(score, 3))
    
    
    

1 	 Kevin Durant 	 0.642
2 	 Stephen Curry 	 0.565
3 	 Giannis Antetokounmpo 	 0.552
4 	 Kawhi Leonard 	 0.545
5 	 Jayson Tatum 	 0.514
6 	 Khris Middleton 	 0.481
7 	 LeBron James 	 0.46
8 	 Pascal Siakam 	 0.451
9 	 Anthony Davis 	 0.443
10 	 Paul George 	 0.436
11 	 Jusuf Nurkić 	 0.433
12 	 Rudy Gobert 	 0.407
13 	 Gordon Hayward 	 0.397
14 	 Jimmy Butler 	 0.397
15 	 Kyle Lowry 	 0.392
16 	 Bam Adebayo 	 0.39
17 	 Chris Paul 	 0.383
18 	 James Harden 	 0.381
19 	 P.J. Tucker 	 0.373
20 	 Danny Green 	 0.369
21 	 Duncan Robinson 	 0.369
22 	 Nikola Jokić 	 0.366
23 	 Brook Lopez 	 0.364
24 	 Marcus Smart 	 0.359
25 	 Kristaps Porziņģis 	 0.356
26 	 Joel Embiid 	 0.352
27 	 Eric Bledsoe 	 0.342
28 	 Joe Ingles 	 0.328
29 	 Jamal Murray 	 0.328
30 	 Paul Millsap 	 0.326
31 	 Clint Capela 	 0.319
32 	 Ivica Zubac 	 0.317
33 	 Jrue Holiday 	 0.317
34 	 Dorian Finney-Smith 	 0.315
35 	 Draymond Green 	 0.308
36 	 Russell Westbrook 	 0.301
37 	 Jaylen Brown 	 0.3
38 	 Patrick Beverley 	 

**SIMULATING SEASON

In [1414]:
def get_seed(conference,sorted_teams):
    counter = 0
    for i in range(len(sorted_teams)):
        team = sorted_teams[i]
        if team in conference: 
            counter+=1
            if counter == 1:
                oneseed = team
            if counter == 2:
                twoseed = team
            if counter == 3:
                threeseed = team
            if counter == 4:
                fourseed = team
            if counter == 5:
                fiveseed = team
            if counter == 6:
                sixseed = team
            if counter == 7:
                sevenseed = team
            if counter == 8:
                eightseed = team
            if counter == 9:
                nineseed = team
            if counter == 10:
                tenseed = team
    return oneseed, twoseed, threeseed, fourseed, fiveseed, sixseed, sevenseed, eightseed, nineseed, tenseed

In [1415]:
def playingames(sevenseed, eightseed, nineseed, tenseed):
    proj=pc_model([projected_abilities[sevenseed]],[projected_abilities[eightseed]])
    if random.randrange(0,100,1)/100<proj[0,0]:
        efsevenseed = sevenseed
        loserseed = eightseed
    else:
        efsevenseed = eightseed
        loserseed = sevenseed


    #play-in game two    
    proj=pc_model([projected_abilities[nineseed]],[projected_abilities[tenseed]])
    if random.randrange(0,100,1)/100<proj[0,0]:
        winnerseed = nineseed
    else:
        winnerseed = tenseed

    #play-in game three
    proj=pc_model([projected_abilities[loserseed]],[projected_abilities[winnerseed]])
    if random.randrange(0,100,1)/100<proj[0,0]:
        efeightseed = loserseed
    else:
        efeightseed = winnerseed
    return efsevenseed, efeightseed

In [1416]:
def matchup(teamone, teamtwo):
    #first matchup
    proj=pc_model([projected_abilities[teamone]],[projected_abilities[teamtwo]])
    probwin = proj[0,0]
    probloss = 1-probwin
    seriesprob = 35*(probwin**4)*(probloss**3)+21*(probwin**5)*(probloss**2)+7*(probwin**6)*(probloss)+probwin**7
    if random.randrange(0,100,1)/100<seriesprob:
        winner = teamone
    else:
        winner = teamtwo
    return winner



In [1417]:
def playoffgames(oneseed, twoseed, threeseed, fourseed, fiveseed, sixseed, sevenseed, eightseed):
    winner1 = matchup(oneseed, eightseed)
    winner2 = matchup(twoseed, sevenseed)
    winner3 = matchup(threeseed, sixseed)
    winner4 = matchup(fourseed, fiveseed)
    
    winnercf1 = matchup(winner1, winner4)
    winnercf2 = matchup(winner2, winner3)
    
    finalist = matchup(winnercf1, winnercf2)
    
    return finalist

In [1591]:
#SIMULATE MIDWAY THROUGH
projected_abilities = {}
team_abilities = {}
for team in NBA:
    team_list = []
    for player in filter(lambda name: name in heightweight[team][2021], embeddings):
        minutes_tensor = torch.tensor(mp[player]['projected'], dtype=torch.float32)
        predicted_score = embeddings[player]['2020'].squeeze() if '2020' in embeddings[player] else embeddings[player]['current']
        embedding_plus_minutes = torch.cat([predicted_score, minutes_tensor.unsqueeze(0)], dim=0)
        team_list.append(embedding_plus_minutes.float().detach())
    projected_abilities[team] = team_list
    team_abilities[team] = pc_model.find_team_embedding(team_list)

team="MIA"
opponent="LAL"
proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
proj

tensor([[0.4973, 0.5027]], grad_fn=<StackBackward>)

In [1588]:
#NOW TO SIMULATE SOME PLAYOFFS IF STARTING FROM SCRATCH
tot=0
winsim={}
playoffs={}
finals={}
champions={}

for team in NBA:
    playoffs[team]=0
    finals[team]=0
    champions[team]=0
    
#SIMULATE MIDWAY THROUGH
projected_abilities = {}
team_abilities = {}
for team in NBA:
    team_list = []
    for player in filter(lambda name: name in heightweight[team][2021], embeddings):
        minutes_tensor = torch.tensor(mp[player]['projected'], dtype=torch.float32)
        predicted_score = embeddings[player]['predicted'].squeeze() if 'predicted' in embeddings[player] else embeddings[player]['current']
        embedding_plus_minutes = torch.cat([predicted_score, minutes_tensor.unsqueeze(0)], dim=0)
        team_list.append(embedding_plus_minutes.float().detach())
    projected_abilities[team] = team_list
    team_abilities[team] = pc_model.find_team_embedding(team_list)
    
#GENERATE POTENTIAL MATCHUPS
pot_matchups = {}
counter = 0
iterator = 0
incount={}
for team in NBA:
    incount[team]=iterator
    iterator+=1
for team in NBA:
    for opponent in NBA:
        if incount[team]<incount[opponent]:
            idval=team+opponent
            if team in East and opponent in East:
                maxval = 3
            if team in West and opponent in West:
                maxval = 3
            if team in East and opponent in West:
                maxval = 2 #there cannot be team in West and opp in East because the East teams go first 
            pot_matchups[idval]={'played': 0, 'max': maxval}
            counter+=1
    
#GET REMAINING MATCHUPS
curr_season = gameresults[7702:len(gameresults),:]

for i in range(len(curr_season)):
    team1=team_to_abbr_mapping[curr_season[i,1]]
    team2=team_to_abbr_mapping[curr_season[i,2]]
    
    if incount[team1]<incount[team2]:
        idval=team1+team2
    else:
        idval=team2+team1
    pot_matchups[idval]['played']+=0
    
#get current wins


for simnum in range(10000):
    for team in NBA:
        winsim[team]=0
    for i in range(len(curr_season)):
        winner_abbr=team_to_abbr_mapping[curr_season[i,1]]
        winsim[winner_abbr]+=1
    for gamename in pot_matchups:
        diff = pot_matchups[gamename]['max']-pot_matchups[gamename]['played']
        team = re.search('^.{3}',gamename).group(0)
        opponent = re.search('.{3}$',gamename).group(0)
        for rep in range(diff):
            proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
            if random.random()<proj[0,0]:
                winsim[team]+=1
            else:
                winsim[opponent]+=1

    sorted_teams = sorted([team for team in winsim], key=lambda x: winsim[x], reverse=True)

    eastc=0
    westc=0
    for i in range(30):
        team = sorted_teams[i]
        if team in East:
            eastc+=1
            if eastc <= 6:
                playoffs[team]+=1                

        if team in West:
            westc+=1
            if westc <= 6:
                playoffs[team]+=1


    #EAST playoffs
    eoneseed, etwoseed, ethreeseed, efourseed, efiveseed, esixseed, esevenseed, eeightseed, enineseed, etenseed = get_seed(East,sorted_teams) 
    esevenseed, eeightseed = playingames(esevenseed, eeightseed, enineseed, etenseed)
    playoffs[esevenseed]+=1
    playoffs[eeightseed]+=1

    efinalist = playoffgames(eoneseed, etwoseed, ethreeseed, efourseed, efiveseed, esixseed, esevenseed, eeightseed)
    finals[efinalist]+=1

    #WEST playoffs
    woneseed, wtwoseed, wthreeseed, wfourseed, wfiveseed, wsixseed, wsevenseed, weightseed, wnineseed, wtenseed = get_seed(West,sorted_teams) 
    wsevenseed, weightseed = playingames(wsevenseed, weightseed, wnineseed, wtenseed)
    playoffs[wsevenseed]+=1
    playoffs[weightseed]+=1

    wfinalist = playoffgames(woneseed, wtwoseed, wthreeseed, wfourseed, wfiveseed, wsixseed, wsevenseed, weightseed)
    finals[wfinalist]+=1

    #FINALS
    champion=matchup(efinalist,wfinalist)
    champions[champion]+=1

    print(simnum, champion, champions[champion])

df_p = pd.DataFrame(list(playoffs.items()),columns = ['team','playoffs']) 
df_f = pd.DataFrame(list(finals.items()),columns = ['team','finals']) 
df_c = pd.DataFrame(list(champions.items()),columns = ['team','champions']) 

df_p.to_csv("/Users/williamarnesen/Desktop/NBA_Model/playoffsprob_init.csv")
df_f.to_csv("/Users/williamarnesen/Desktop/NBA_Model/finalsprob_init.csv")
df_c.to_csv("/Users/williamarnesen/Desktop/NBA_Model/championprob_init.csv")
    


0 MIL 1
1 LAL 1
2 MIL 2
3 MIA 1
4 LAL 2
5 LAL 3
6 MIA 2
7 LAL 4
8 PHI 1
9 DEN 1
10 DEN 2
11 DEN 3
12 BOS 1
13 POR 1
14 BOS 2
15 LAL 5
16 MIA 3
17 PHO 1
18 BOS 3
19 LAC 1
20 LAC 2
21 MIA 4
22 LAL 6
23 PHO 2
24 DEN 4
25 TOR 1
26 MIL 3
27 TOR 2
28 LAC 3
29 POR 2
30 MIA 5
31 LAC 4
32 PHI 2
33 PHI 3
34 DEN 5
35 LAC 5
36 MIA 6
37 MIA 7
38 LAL 7
39 MIL 4
40 LAL 8
41 LAL 9
42 LAC 6
43 MIA 8
44 BOS 4
45 LAC 7
46 MIA 9
47 LAC 8
48 DEN 6
49 LAL 10
50 MIA 10
51 TOR 3
52 GSW 1
53 MIA 11
54 DEN 7
55 BOS 5
56 MIA 12
57 POR 3
58 LAL 11
59 LAC 9
60 LAC 10
61 LAC 11
62 LAL 12
63 PHI 4
64 TOR 4
65 BOS 6
66 MIA 13
67 BOS 7
68 IND 1
69 LAC 12
70 LAC 13
71 MIA 14
72 DEN 8
73 MIL 5
74 PHI 5
75 UTA 1
76 LAL 13
77 MIA 15
78 PHO 3
79 MIA 16
80 BOS 8
81 MIA 17
82 MIA 18
83 LAC 14
84 PHI 6
85 MIA 19
86 TOR 5
87 HOU 1
88 LAC 15
89 LAL 14
90 LAC 16
91 DEN 9
92 MIA 20
93 LAL 15
94 LAC 17
95 LAC 18
96 LAC 19
97 LAC 20
98 DEN 10
99 BOS 9
100 DEN 11
101 PHI 7
102 LAC 21
103 MIA 21
104 LAC 22
105 LAL 16
106 PHO 4
107 UT

764 HOU 22
765 LAC 134
766 LAC 135
767 HOU 23
768 BOS 45
769 BOS 46
770 MIA 115
771 DEN 98
772 DEN 99
773 BOS 47
774 LAC 136
775 BOS 48
776 MIL 73
777 LAC 137
778 NOP 4
779 LAL 93
780 LAC 138
781 LAC 139
782 DEN 100
783 MIA 116
784 MIA 117
785 PHI 62
786 LAL 94
787 DEN 101
788 LAL 95
789 DEN 102
790 DEN 103
791 LAC 140
792 LAL 96
793 BOS 49
794 MIL 74
795 POR 24
796 LAL 97
797 BOS 50
798 MIA 118
799 LAC 141
800 DEN 104
801 MIA 119
802 LAL 98
803 MIL 75
804 DEN 105
805 MIA 120
806 LAL 99
807 TOR 34
808 MIA 121
809 MIA 122
810 MIL 76
811 DEN 106
812 DEN 107
813 LAL 100
814 TOR 35
815 DAL 6
816 LAC 142
817 MIA 123
818 PHI 63
819 TOR 36
820 LAC 143
821 POR 25
822 TOR 37
823 LAC 144
824 DEN 108
825 POR 26
826 PHO 24
827 MIA 124
828 GSW 5
829 DEN 109
830 LAC 145
831 POR 27
832 BOS 51
833 PHO 25
834 MIL 77
835 HOU 24
836 MIL 78
837 BRK 10
838 DAL 7
839 MIA 125
840 MIA 126
841 MIL 79
842 LAL 101
843 DEN 110
844 MIL 80
845 LAC 146
846 LAL 102
847 PHO 26
848 TOR 38
849 SAS 14
850 TOR 39
851 MIA 

1434 PHO 46
1435 DEN 173
1436 LAC 250
1437 BOS 88
1438 UTA 25
1439 BOS 89
1440 MIL 130
1441 BOS 90
1442 PHI 97
1443 PHI 98
1444 DEN 174
1445 LAC 251
1446 MIL 131
1447 MIA 204
1448 BOS 91
1449 MIA 205
1450 MIA 206
1451 LAL 197
1452 MIL 132
1453 PHI 99
1454 DEN 175
1455 PHO 47
1456 MIA 207
1457 UTA 26
1458 PHO 48
1459 SAS 25
1460 DEN 176
1461 MIL 133
1462 LAC 252
1463 MIA 208
1464 MIL 134
1465 MIA 209
1466 PHI 100
1467 LAC 253
1468 LAL 198
1469 TOR 66
1470 LAL 199
1471 LAL 200
1472 MIL 135
1473 MIA 210
1474 TOR 67
1475 PHI 101
1476 BOS 92
1477 MIA 211
1478 DEN 177
1479 DEN 178
1480 LAC 254
1481 LAC 255
1482 MIA 212
1483 LAL 201
1484 PHI 102
1485 LAL 202
1486 DEN 179
1487 MIL 136
1488 LAL 203
1489 LAL 204
1490 DAL 13
1491 MIA 213
1492 DEN 180
1493 PHO 49
1494 TOR 68
1495 DEN 181
1496 UTA 27
1497 LAL 205
1498 DEN 182
1499 MIA 214
1500 HOU 43
1501 PHO 50
1502 MIA 215
1503 TOR 69
1504 HOU 44
1505 LAC 256
1506 DEN 183
1507 MIA 216
1508 UTA 28
1509 TOR 70
1510 POR 47
1511 DEN 184
1512 PHI 103


2076 PHI 149
2077 LAL 295
2078 MIL 188
2079 PHI 150
2080 POR 68
2081 TOR 95
2082 LAC 357
2083 LAC 358
2084 TOR 96
2085 DEN 247
2086 MIA 297
2087 PHI 151
2088 LAC 359
2089 HOU 54
2090 LAC 360
2091 PHI 152
2092 MIA 298
2093 LAC 361
2094 LAC 362
2095 GSW 8
2096 DEN 248
2097 HOU 55
2098 DAL 16
2099 BOS 127
2100 BOS 128
2101 DEN 249
2102 MIA 299
2103 LAL 296
2104 LAC 363
2105 LAC 364
2106 PHO 63
2107 LAL 297
2108 PHI 153
2109 DEN 250
2110 TOR 97
2111 PHI 154
2112 HOU 56
2113 PHI 155
2114 MIA 300
2115 BOS 129
2116 MIA 301
2117 MIL 189
2118 LAC 365
2119 PHI 156
2120 LAC 366
2121 BOS 130
2122 MIA 302
2123 DEN 251
2124 PHI 157
2125 BOS 131
2126 UTA 46
2127 LAL 298
2128 BOS 132
2129 MIA 303
2130 TOR 98
2131 LAC 367
2132 PHI 158
2133 DAL 17
2134 POR 69
2135 PHI 159
2136 LAC 368
2137 MIA 304
2138 PHO 64
2139 TOR 99
2140 DEN 252
2141 BOS 133
2142 MIA 305
2143 DEN 253
2144 MIA 306
2145 TOR 100
2146 MIL 190
2147 LAC 369
2148 MIA 307
2149 MIA 308
2150 LAC 370
2151 DEN 254
2152 MIA 309
2153 LAC 371
215

2716 LAC 463
2717 MIA 405
2718 LAC 464
2719 LAL 365
2720 LAC 465
2721 MIA 406
2722 MIL 241
2723 DEN 319
2724 DEN 320
2725 MIA 407
2726 PHI 201
2727 LAC 466
2728 UTA 56
2729 BRK 37
2730 MIL 242
2731 TOR 135
2732 MIA 408
2733 PHI 202
2734 MIA 409
2735 MIL 243
2736 PHI 203
2737 MIL 244
2738 LAC 467
2739 LAC 468
2740 DEN 321
2741 LAC 469
2742 POR 88
2743 DEN 322
2744 PHO 81
2745 MIA 410
2746 LAC 470
2747 LAL 366
2748 MIA 411
2749 MIA 412
2750 TOR 136
2751 PHI 204
2752 LAC 471
2753 TOR 137
2754 BOS 165
2755 TOR 138
2756 SAS 36
2757 PHO 82
2758 UTA 57
2759 HOU 65
2760 LAL 367
2761 PHO 83
2762 LAC 472
2763 TOR 139
2764 BRK 38
2765 POR 89
2766 LAC 473
2767 PHO 84
2768 LAC 474
2769 BOS 166
2770 MIL 245
2771 LAC 475
2772 LAC 476
2773 MIA 413
2774 LAL 368
2775 DEN 323
2776 DEN 324
2777 MIA 414
2778 LAL 369
2779 LAC 477
2780 MIA 415
2781 MIA 416
2782 MIL 246
2783 TOR 140
2784 LAC 478
2785 BOS 167
2786 LAL 370
2787 MIA 417
2788 LAC 479
2789 SAS 37
2790 BOS 168
2791 MIA 418
2792 DEN 325
2793 LAL 371

...

7136 MIA 1085
7137 SAC 1
7138 DEN 818
7139 LAL 943
7140 HOU 166
7141 LAC 1233
7142 BOS 425
7143 PHI 494
7144 PHO 197
7145 UTA 178
7146 POR 228
7147 LAL 944
7148 BRK 109
7149 PHI 495
7150 TOR 373
7151 LAC 1234
7152 POR 229
7153 LAC 1235
7154 MIL 605
7155 PHI 496
7156 PHI 497
7157 MIA 1086
7158 MIA 1087
7159 MIL 606
7160 MIL 607
7161 NOP 27
7162 BOS 426
7163 BOS 427
7164 LAL 945
7165 LAL 946
7166 LAC 1236
7167 LAL 947
7168 MIL 608
7169 TOR 374
7170 MIL 609
7171 DEN 819
7172 LAL 948
7173 BOS 428
7174 MIA 1088
7175 PHI 498
7176 PHO 198
7177 MIA 1089
7178 LAC 1237
7179 MIA 1090
7180 BRK 110
7181 MIA 1091
7182 DEN 820
7183 MIL 610
7184 BOS 429
7185 MIA 1092
7186 MIA 1093
7187 DEN 821
7188 MIA 1094
7189 MIA 1095
7190 LAC 1238
7191 LAC 1239
7192 LAC 1240
7193 BRK 111
7194 TOR 375
7195 MIL 611
7196 UTA 179
7197 LAC 1241
7198 POR 230
7199 LAC 1242
7200 PHO 199
7201 MIA 1096
7202 UTA 180
7203 MIA 1097
7204 DEN 822
7205 PHO 200
7206 POR 231
7207 LAL 949
7208 MIA 1098
7209 LAC 1243
7210 LAC 1244
72

7750 TOR 405
7751 LAC 1344
7752 MIA 1190
7753 HOU 177
7754 DEN 871
7755 DAL 90
7756 BOS 459
7757 DEN 872
7758 LAC 1345
7759 LAC 1346
7760 POR 247
7761 HOU 178
7762 DEN 873
7763 MIA 1191
7764 DEN 874
7765 PHI 531
7766 PHO 220
7767 BOS 460
7768 LAL 1023
7769 SAS 92
7770 PHO 221
7771 LAC 1347
7772 LAL 1024
7773 MIL 658
7774 MIA 1192
7775 MIL 659
7776 DEN 875
7777 POR 248
7778 LAC 1348
7779 LAL 1025
7780 BOS 461
7781 LAC 1349
7782 LAC 1350
7783 MIA 1193
7784 MIA 1194
7785 DEN 876
7786 POR 249
7787 LAC 1351
7788 NOP 32
7789 LAL 1026
7790 LAC 1352
7791 LAC 1353
7792 DEN 877
7793 LAC 1354
7794 PHO 222
7795 LAL 1027
7796 POR 250
7797 MIA 1195
7798 MIL 660
7799 MIL 661
7800 MIA 1196
7801 MIA 1197
7802 MIA 1198
7803 MIA 1199
7804 TOR 406
7805 DEN 878
7806 BOS 462
7807 MIL 662
7808 MIA 1200
7809 LAC 1355
7810 DEN 879
7811 DEN 880
7812 MIL 663
7813 PHO 223
7814 MIA 1201
7815 MIA 1202
7816 DEN 881
7817 MIA 1203
7818 LAL 1028
7819 BRK 115
7820 UTA 202
7821 LAC 1356
7822 MIA 1204
7823 MIL 664
7824 LA

8361 DEN 941
8362 MIA 1286
8363 MIA 1287
8364 DEN 942
8365 LAL 1110
8366 LAC 1443
8367 LAC 1444
8368 BOS 502
8369 PHI 571
8370 LAC 1445
8371 PHO 239
8372 POR 262
8373 LAL 1111
8374 TOR 438
8375 SAS 104
8376 LAL 1112
8377 MIL 701
8378 DEN 943
8379 MIA 1288
8380 BRK 123
8381 UTA 213
8382 GSW 33
8383 TOR 439
8384 LAL 1113
8385 MIA 1289
8386 PHI 572
8387 MIL 702
8388 UTA 214
8389 BOS 503
8390 DEN 944
8391 LAL 1114
8392 MIL 703
8393 MIL 704
8394 MIL 705
8395 HOU 192
8396 LAC 1446
8397 LAL 1115
8398 MIA 1290
8399 LAL 1116
8400 DEN 945
8401 TOR 440
8402 MIL 706
8403 MIL 707
8404 DEN 946
8405 LAC 1447
8406 LAC 1448
8407 DEN 947
8408 MIA 1291
8409 LAC 1449
8410 LAC 1450
8411 MIA 1292
8412 HOU 193
8413 LAC 1451
8414 DAL 100
8415 LAL 1117
8416 LAC 1452
8417 BOS 504
8418 BRK 124
8419 MIA 1293
8420 LAL 1118
8421 POR 263
8422 LAC 1453
8423 MIL 708
8424 DEN 948
8425 MIL 709
8426 BOS 505
8427 TOR 441
8428 BOS 506
8429 LAC 1454
8430 DEN 949
8431 DEN 950
8432 LAC 1455
8433 LAC 1456
8434 PHO 240
8435 BOS

8971 LAL 1187
8972 BRK 135
8973 LAC 1546
8974 MIL 755
8975 LAC 1547
8976 DEN 1014
8977 PHI 606
8978 MIL 756
8979 PHO 251
8980 MIA 1373
8981 PHI 607
8982 MIA 1374
8983 LAC 1548
8984 SAS 111
8985 TOR 474
8986 TOR 475
8987 LAC 1549
8988 MIL 757
8989 BOS 547
8990 DEN 1015
8991 PHI 608
8992 LAC 1550
8993 DEN 1016
8994 MIL 758
8995 LAC 1551
8996 DEN 1017
8997 DEN 1018
8998 PHI 609
8999 PHI 610
9000 LAC 1552
9001 DEN 1019
9002 BOS 548
9003 MIA 1375
9004 LAC 1553
9005 LAL 1188
9006 BOS 549
9007 DEN 1020
9008 LAC 1554
9009 LAL 1189
9010 MIA 1376
9011 HOU 205
9012 LAC 1555
9013 MIA 1377
9014 POR 278
9015 LAL 1190
9016 LAL 1191
9017 MIL 759
9018 LAL 1192
9019 MIA 1378
9020 BOS 550
9021 MIA 1379
9022 MIL 760
9023 LAC 1556
9024 LAC 1557
9025 BRK 136
9026 LAC 1558
9027 LAC 1559
9028 LAC 1560
9029 DEN 1021
9030 MIL 761
9031 POR 279
9032 DAL 111
9033 BOS 551
9034 LAL 1193
9035 DEN 1022
9036 LAC 1561
9037 LAC 1562
9038 HOU 206
9039 MIA 1380
9040 DEN 1023
9041 POR 280
9042 PHI 611
9043 PHO 252
9044 LAC 

9576 MIA 1450
9577 MIL 800
9578 LAC 1660
9579 BRK 144
9580 BOS 585
9581 DAL 118
9582 LAC 1661
9583 DEN 1094
9584 DEN 1095
9585 TOR 509
9586 MIL 801
9587 MIA 1451
9588 BRK 145
9589 MIA 1452
9590 LAC 1662
9591 TOR 510
9592 PHI 646
9593 LAC 1663
9594 LAL 1261
9595 LAC 1664
9596 PHI 647
9597 PHI 648
9598 SAS 120
9599 PHI 649
9600 UTA 240
9601 POR 297
9602 LAC 1665
9603 MIL 802
9604 TOR 511
9605 DEN 1096
9606 MIA 1453
9607 BOS 586
9608 MIA 1454
9609 LAL 1262
9610 MIA 1455
9611 LAC 1666
9612 MIL 803
9613 PHI 650
9614 PHI 651
9615 MIA 1456
9616 MIL 804
9617 MIL 805
9618 MIL 806
9619 DEN 1097
9620 NOP 40
9621 MIL 807
9622 LAC 1667
9623 BOS 587
9624 LAL 1263
9625 MIL 808
9626 MIL 809
9627 PHI 652
9628 GSW 37
9629 LAL 1264
9630 LAC 1668
9631 BOS 588
9632 LAL 1265
9633 DEN 1098
9634 HOU 222
9635 DEN 1099
9636 PHO 277
9637 MIL 810
9638 BOS 589
9639 MIL 811
9640 BOS 590
9641 PHI 653
9642 LAC 1669
9643 DEN 1100
9644 MIA 1457
9645 DEN 1101
9646 LAC 1670
9647 SAS 121
9648 MIL 812
9649 DEN 1102
9650 LA

In [1587]:
tot=0
winsim={}
playoffs={}
finals={}
champions={}

for team in NBA:
    playoffs[team]=0
    finals[team]=0
    champions[team]=0
    
#SIMULATE MIDWAY THROUGH
projected_abilities = {}
team_abilities = {}
for team in NBA:
    team_list = []
    for player in filter(lambda name: name in heightweight[team][2021], embeddings):
        minutes_tensor = torch.tensor(mp[player]['projected'], dtype=torch.float32)
        predicted_score = embeddings[player]['current'].squeeze() if 'current' in embeddings[player] else embeddings[player]['current']
        embedding_plus_minutes = torch.cat([predicted_score, minutes_tensor.unsqueeze(0)], dim=0)
        team_list.append(embedding_plus_minutes.float().detach())
    projected_abilities[team] = team_list
    team_abilities[team] = pc_model.find_team_embedding(team_list)
    
#GENERATE POTENTIAL MATCHUPS
pot_matchups = {}
counter = 0
iterator = 0
incount={}
for team in NBA:
    incount[team]=iterator
    iterator+=1
for team in NBA:
    for opponent in NBA:
        if incount[team]<incount[opponent]:
            idval=team+opponent
            if team in East and opponent in East:
                maxval = 3
            if team in West and opponent in West:
                maxval = 3
            if team in East and opponent in West:
                maxval = 2 #there cannot be team in West and opp in East because the East teams go first 
            pot_matchups[idval]={'played': 0, 'max': maxval}
            counter+=1
    
#GET REMAINING MATCHUPS
curr_season = gameresults[7702:len(gameresults),:]

for i in range(len(curr_season)):
    team1=team_to_abbr_mapping[curr_season[i,1]]
    team2=team_to_abbr_mapping[curr_season[i,2]]
    
    if incount[team1]<incount[team2]:
        idval=team1+team2
    else:
        idval=team2+team1
    pot_matchups[idval]['played']+=1
    
#get current wins


for simnum in range(2000):
    for team in NBA:
        winsim[team]=0
    for i in range(len(curr_season)):
        winner_abbr=team_to_abbr_mapping[curr_season[i,1]]
        winsim[winner_abbr]+=1
    for gamename in pot_matchups:
        diff = pot_matchups[gamename]['max']-pot_matchups[gamename]['played']
        team = re.search('^.{3}',gamename).group(0)
        opponent = re.search('.{3}$',gamename).group(0)
        for rep in range(diff):
            proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
            if random.random()<proj[0,0]:
                winsim[team]+=1
            else:
                winsim[opponent]+=1

    sorted_teams = sorted([team for team in winsim], key=lambda x: winsim[x], reverse=True)

    eastc=0
    westc=0
    for i in range(30):
        team = sorted_teams[i]
        if team in East:
            eastc+=1
            if eastc <= 6:
                playoffs[team]+=1                

        if team in West:
            westc+=1
            if westc <= 6:
                playoffs[team]+=1


    #EAST playoffs
    eoneseed, etwoseed, ethreeseed, efourseed, efiveseed, esixseed, esevenseed, eeightseed, enineseed, etenseed = get_seed(East,sorted_teams) 
    esevenseed, eeightseed = playingames(esevenseed, eeightseed, enineseed, etenseed)
    playoffs[esevenseed]+=1
    playoffs[eeightseed]+=1

    efinalist = playoffgames(eoneseed, etwoseed, ethreeseed, efourseed, efiveseed, esixseed, esevenseed, eeightseed)
    finals[efinalist]+=1

    #WEST playoffs
    woneseed, wtwoseed, wthreeseed, wfourseed, wfiveseed, wsixseed, wsevenseed, weightseed, wnineseed, wtenseed = get_seed(West,sorted_teams) 
    wsevenseed, weightseed = playingames(wsevenseed, weightseed, wnineseed, wtenseed)
    playoffs[wsevenseed]+=1
    playoffs[weightseed]+=1

    wfinalist = playoffgames(woneseed, wtwoseed, wthreeseed, wfourseed, wfiveseed, wsixseed, wsevenseed, weightseed)
    finals[wfinalist]+=1

    #FINALS
    champion=matchup(efinalist,wfinalist)
    champions[champion]+=1

    print(simnum, champion, champions[champion])
    
df_p = pd.DataFrame(list(playoffs.items()),columns = ['team','playoffs']) 
df_f = pd.DataFrame(list(finals.items()),columns = ['team','finals']) 
df_c = pd.DataFrame(list(champions.items()),columns = ['team','champions']) 

df_p.to_csv("/Users/williamarnesen/Desktop/NBA_Model/playoffsprob_init.csv")
df_f.to_csv("/Users/williamarnesen/Desktop/NBA_Model/finalsprob_init.csv")
df_c.to_csv("/Users/williamarnesen/Desktop/NBA_Model/championprob_init.csv")


0 LAL 1
1 DEN 1
2 DEN 2
3 DEN 3
4 POR 1
5 TOR 1
6 DEN 4
7 LAL 2
8 PHO 1
9 MIL 1
10 LAC 1
11 LAC 2
12 LAL 3
13 LAC 3
14 HOU 1
15 LAC 4
16 LAC 5
17 DEN 5
18 DAL 1
19 LAL 4
20 LAC 6
21 TOR 2
22 LAC 7
23 BOS 1
24 PHI 1
25 LAC 8
26 MIA 1
27 MIL 2
28 MIA 2
29 MIL 3
30 DEN 6
31 LAL 5
32 BOS 2
33 LAC 9
34 TOR 3
35 LAL 6
36 HOU 2
37 LAL 7
38 HOU 3
39 DEN 7
40 MIL 4
41 MIL 5
42 MIA 3
43 LAC 10
44 LAC 11
45 LAC 12
46 BOS 3
47 MIA 4
48 LAC 13
49 LAL 8
50 LAC 14
51 LAC 15
52 POR 2
53 PHI 2
54 MIL 6
55 MIA 5
56 LAC 16
57 DEN 8
58 UTA 1
59 MIA 6
60 UTA 2
61 MIA 7
62 DEN 9
63 MIL 7
64 LAC 17
65 LAL 9
66 DEN 10
67 DEN 11
68 PHI 3
69 DEN 12
70 MIA 8
71 TOR 4
72 LAL 10
73 LAL 11
74 LAL 12
75 LAC 18
76 BOS 4
77 UTA 3
78 POR 3
79 HOU 4
80 MIA 9
81 DEN 13
82 MIA 10
83 TOR 5
84 DEN 14
85 LAC 19
86 TOR 6
87 POR 4
88 DEN 15
89 DEN 16
90 LAL 13
91 DEN 17
92 ATL 1
93 MIL 8
94 PHO 2
95 PHI 4
96 LAL 14
97 BOS 5
98 MIL 9
99 LAC 20
100 MIA 11
101 DEN 18
102 HOU 5
103 LAL 15
104 BOS 6
105 DEN 19
106 PHI 5
107 LAL 16


760 PHI 51
761 MIL 97
762 PHO 22
763 LAC 163
764 ATL 11
765 LAL 136
766 LAC 164
767 MIA 35
768 BOS 40
769 PHI 52
770 SAS 6
771 LAL 137
772 SAS 7
773 LAC 165
774 MIL 98
775 LAC 166
776 LAL 138
777 BOS 41
778 PHI 53
779 MIL 99
780 LAC 167
781 LAC 168
782 MIA 36
783 LAL 139
784 BRK 19
785 BOS 42
786 MIA 37
787 LAC 169
788 PHO 23
789 DEN 97
790 MIL 100
791 DEN 98
792 LAC 170
793 LAL 140
794 DEN 99
795 PHO 24
796 BOS 43
797 HOU 19
798 LAL 141
799 DEN 100
800 POR 18
801 BOS 44
802 DEN 101
803 MIL 101
804 MIL 102
805 MIL 103
806 LAC 171
807 BRK 20
808 POR 19
809 BOS 45
810 MIL 104
811 LAC 172
812 MIL 105
813 MIL 106
814 LAC 173
815 BOS 46
816 HOU 20
817 POR 20
818 MIL 107
819 PHI 54
820 DEN 102
821 DEN 103
822 MIL 108
823 DEN 104
824 MIA 38
825 POR 21
826 LAC 174
827 LAC 175
828 TOR 24
829 LAL 142
830 WAS 5
831 LAC 176
832 LAC 177
833 DEN 105
834 MIL 109
835 LAL 143
836 MIA 39
837 LAC 178
838 MIL 110
839 DEN 106
840 MIL 111
841 DEN 107
842 MIA 40
843 MIL 112
844 BRK 21
845 PHI 55
846 LAC 179


1429 LAL 243
1430 HOU 32
1431 LAL 244
1432 DEN 199
1433 LAL 245
1434 PHI 96
1435 LAC 289
1436 LAL 246
1437 HOU 33
1438 LAC 290
1439 LAC 291
1440 MIL 192
1441 LAC 292
1442 LAC 293
1443 BOS 69
1444 LAC 294
1445 PHO 38
1446 BOS 70
1447 LAL 247
1448 HOU 34
1449 BRK 34
1450 ATL 17
1451 DEN 200
1452 PHI 97
1453 LAL 248
1454 LAL 249
1455 LAC 295
1456 DEN 201
1457 LAC 296
1458 LAL 250
1459 MIL 193
1460 LAL 251
1461 MIA 73
1462 BOS 71
1463 LAC 297
1464 MIL 194
1465 MIL 195
1466 LAL 252
1467 IND 16
1468 MIA 74
1469 POR 42
1470 DEN 202
1471 BRK 35
1472 BOS 72
1473 HOU 35
1474 PHO 39
1475 PHI 98
1476 LAL 253
1477 BRK 36
1478 LAL 254
1479 PHI 99
1480 LAL 255
1481 LAC 298
1482 DEN 203
1483 MIL 196
1484 MIL 197
1485 PHI 100
1486 LAL 256
1487 DEN 204
1488 LAC 299
1489 BOS 73
1490 MIL 198
1491 PHO 40
1492 BOS 74
1493 MIL 199
1494 LAL 257
1495 LAC 300
1496 LAC 301
1497 MIA 75
1498 LAC 302
1499 DEN 205
1500 LAC 303
1501 POR 43
1502 LAL 258
1503 LAC 304
1504 DEN 206
1505 MIL 200
1506 MIL 201
1507 TOR 43
1

In [1584]:
team="MIL"
opponent="DEN"
proj=pc_model([projected_abilities[team]],[projected_abilities[opponent]])
proj

tensor([[0.4790, 0.5210]], grad_fn=<StackBackward>)

In [1571]:
sorted_list = sorted([(team, playoffs[team]) for team in playoffs], key=lambda x: playoffs[x[0]], reverse=True)
for i, (team, prob) in enumerate(sorted_list):
    print(i + 1, team, prob/1000)

1 MIL 0.991
2 LAL 0.991
3 LAC 0.991
4 PHI 0.98
5 DEN 0.97
6 MIA 0.948
7 BOS 0.948
8 TOR 0.87
9 PHO 0.865
10 BRK 0.841
11 ATL 0.809
12 POR 0.808
13 HOU 0.777
14 UTA 0.776
15 IND 0.754
16 DAL 0.5
17 WAS 0.474
18 SAS 0.429
19 NOP 0.309
20 SAC 0.202
21 GSW 0.193
22 ORL 0.175
23 MEM 0.117
24 DET 0.069
25 CLE 0.066
26 CHO 0.041
27 OKC 0.039
28 MIN 0.033
29 NYK 0.032
30 CHI 0.002


In [1572]:
sorted_list = sorted([(team, champions[team]) for team in champions], key=lambda x: champions[x[0]], reverse=True)
for i, (team, prob) in enumerate(sorted_list):
    print(i + 1, team, prob/1000)

1 LAC 0.207
2 LAL 0.171
3 MIL 0.151
4 DEN 0.127
5 PHI 0.062
6 BOS 0.052
7 MIA 0.051
8 TOR 0.033
9 PHO 0.03
10 HOU 0.026
11 POR 0.025
12 BRK 0.02
13 UTA 0.011
14 DAL 0.009
15 ATL 0.007
16 IND 0.006
17 SAS 0.006
18 WAS 0.005
19 NOP 0.001
20 CHI 0.0
21 NYK 0.0
22 DET 0.0
23 CLE 0.0
24 ORL 0.0
25 CHO 0.0
26 MEM 0.0
27 GSW 0.0
28 SAC 0.0
29 OKC 0.0
30 MIN 0.0


In [1573]:
sorted_list = sorted([(team, finals[team]) for team in finals], key=lambda x: finals[x[0]], reverse=True)
for i, (team, prob) in enumerate(sorted_list):
    print(i + 1, team, prob/2000)

1 LAC 0.153
2 MIL 0.147
3 LAL 0.128
4 DEN 0.0985
5 PHI 0.0905
6 BOS 0.072
7 MIA 0.065
8 TOR 0.0425
9 BRK 0.036
10 PHO 0.0325
11 POR 0.03
12 HOU 0.0285
13 ATL 0.0185
14 IND 0.017
15 UTA 0.013
16 WAS 0.011
17 DAL 0.008
18 SAS 0.0065
19 NOP 0.001
20 NYK 0.0005
21 MEM 0.0005
22 GSW 0.0005
23 CHI 0.0
24 DET 0.0
25 CLE 0.0
26 ORL 0.0
27 CHO 0.0
28 SAC 0.0
29 OKC 0.0
30 MIN 0.0


In [1566]:
df_p = pd.DataFrame(list(playoffs.items()),columns = ['team','playoffs']) 
df_f = pd.DataFrame(list(finals.items()),columns = ['team','finals']) 
df_c = pd.DataFrame(list(champions.items()),columns = ['team','champions']) 

df_p.to_csv("/Users/williamarnesen/Desktop/NBA_Model/playoffsprob_init.csv")
df_f.to_csv("/Users/williamarnesen/Desktop/NBA_Model/finalsprob_init.csv")
df_c.to_csv("/Users/williamarnesen/Desktop/NBA_Model/championprob_init.csv")

# **CALCULATING PERFORMANCE DURING YEAR TO DATE

In [1346]:
gamescorespd = pd.read_csv("/Users/williamarnesen/Desktop/NBA_Model/full_sheet_scaled.csv")
gamescores0 = gamescorespd.values
gamescores = copy.deepcopy(gamescores0)

In [1347]:
running = torch.zeros(4, dtype=torch.float32)
for team in NBA:
    running += team_abilities[team]
avg = running/30

all_minutes_tensor = torch.tensor(48, dtype=torch.float32).unsqueeze(0).float()
base_list = [torch.cat([avg, all_minutes_tensor], dim=0).float() for i in range(6)]

def create_generic_team_with_player(player, minutes, description_tensor):
    
    new_list = [element for element in base_list]
    minutes_tensor = torch.tensor(minutes, dtype=torch.float32).unsqueeze(0).float()
    new_list[0] = torch.cat([description_tensor.float(), minutes_tensor], dim=0)

    remaining_minutes_tensor = torch.tensor(48 - minutes, dtype=torch.float32).unsqueeze(0).float()
    new_list[-1] = torch.cat([avg, remaining_minutes_tensor], dim=0).float()
    return new_list if minutes < 48 else new_list[:-1]
    

In [1567]:
#INSEASON JUMP HERE
war = {}
ind_game_war = []

curr_season = gameresults[7702:len(gameresults),:]
list_game_results = list([result[0] for result in gameresults])

for i in range(len(gamescores)):
    game = gamescores[i,40]
    if game in curr_season[:,0]:
        player=gamescores[i,0]
        war[player]=0


marked = set()
with torch.no_grad():
    for i in range(len(gamescores)):
        game = gamescores[i,40]
        player=gamescores[i,0]
        if game in curr_season[:,0] and (game, player) not in marked and gamescores[i, -1] > 0:
    
            marked.add((game, player))
            gameval=gamescores[i,keep].astype('double')
            gamevalinput=torch.from_numpy(gameval)
            
            with torch.no_grad():
                plus_minus = (model2(gamevalinput).item() * gamescores[i, -1])
            
            war[player] += plus_minus
            ind_game_war.append((player, game, plus_minus))
        

sorted_list_curr = sorted([(player, war[player]) for player in war], key=lambda x: war[x[0]], reverse=True)
for i, (player, wins) in enumerate(sorted_list_curr):
    print(i+1,"\t", player, "\t", round(wins, 3))
    

1 	 LeBron James 	 138.705
2 	 Kevin Durant 	 122.039
3 	 Paul George 	 111.154
4 	 Deni Avdija 	 110.948
5 	 Giannis Antetokounmpo 	 110.291
6 	 Trae Young 	 109.227
7 	 Kyrie Irving 	 106.802
8 	 De'Andre Hunter 	 99.441
9 	 Anthony Davis 	 98.18
10 	 Dennis Schröder 	 97.112
11 	 Kawhi Leonard 	 95.808
12 	 Jrue Holiday 	 95.24
13 	 Domantas Sabonis 	 95.147
14 	 Nicolas Batum 	 90.895
15 	 Jamal Murray 	 90.653
16 	 Gary Harris 	 88.192
17 	 Myles Turner 	 87.781
18 	 Patrick Beverley 	 87.415
19 	 Tobias Harris 	 85.181
20 	 John Collins 	 85.096
21 	 Andre Drummond 	 81.685
22 	 Jayson Tatum 	 80.686
23 	 Kentavious Caldwell-Pope 	 78.812
24 	 Cam Reddish 	 75.64
25 	 Jeff Teague 	 74.725
26 	 Joel Embiid 	 74.609
27 	 CJ McCollum 	 70.275
28 	 Steven Adams 	 68.951
29 	 Devonte' Graham 	 68.327
30 	 Mikal Bridges 	 67.866
31 	 Kyle Anderson 	 67.77
32 	 Will Barton 	 67.554
33 	 Victor Oladipo 	 66.967
34 	 Mike Conley 	 66.584
35 	 Terrence Ross 	 64.913
36 	 Bobby Portis 	 62.

408 	 Zion Williamson 	 -47.472
409 	 Kendrick Nunn 	 -49.471
410 	 Jae Crowder 	 -50.22
411 	 Aron Baynes 	 -50.624
412 	 DeMar DeRozan 	 -54.765
413 	 Cedi Osman 	 -55.401
414 	 Tyler Herro 	 -55.69
415 	 J.J. Redick 	 -56.316
416 	 Pascal Siakam 	 -57.122
417 	 Blake Griffin 	 -58.857
418 	 Duncan Robinson 	 -59.224
419 	 Lonnie Walker 	 -60.23
420 	 Théo Maledon 	 -61.925
421 	 Julius Randle 	 -62.811
422 	 Patrick Williams 	 -63.227
423 	 Zach LaVine 	 -63.918
424 	 Reggie Bullock 	 -64.666
425 	 Eric Paschall 	 -69.069
426 	 Terry Rozier 	 -69.761
427 	 Wendell Carter 	 -70.086
428 	 Bam Adebayo 	 -78.676
429 	 Delon Wright 	 -89.217
430 	 Stephen Curry 	 -90.404
431 	 Jimmy Butler 	 -93.638
432 	 D'Angelo Russell 	 -108.547


'201410280LAL'

In [1297]:
sorted_list_temp = sorted(ind_game_war, key=lambda x: x[-1], reverse=True)
for i, (player, game, plus_minus) in enumerate(sorted_list_temp):
    print(i+1,"\t", player, "\t", game, "\t", round(plus_minus, 3))

1 	 Malcolm Brogdon 	 202012310IND 	 53.6
2 	 Jayson Tatum 	 202012290IND 	 51.611
3 	 Robert Covington 	 202101010GSW 	 50.972
4 	 RJ Barrett 	 202012270NYK 	 49.919
5 	 Damian Lillard 	 202101010GSW 	 49.764
6 	 Kevin Durant 	 202012250BOS 	 48.369
7 	 Derrick Jones 	 202101010GSW 	 47.557
8 	 Domantas Sabonis 	 202012310IND 	 46.333
9 	 Coby White 	 202012310WAS 	 45.533
10 	 Shake Milton 	 202012230PHI 	 45.156
11 	 CJ McCollum 	 202101010GSW 	 44.781
12 	 Andrew Wiggins 	 202012290DET 	 43.635
13 	 Kyrie Irving 	 202012250BOS 	 43.476
14 	 Tobias Harris 	 202012260NYK 	 42.929
15 	 Deni Avdija 	 202012270WAS 	 42.596
16 	 Norman Powell 	 202012310TOR 	 42.098
17 	 Kyle Kuzma 	 202012270LAL 	 41.843
18 	 De'Andre Hunter 	 202012230CHI 	 41.674
19 	 LeBron James 	 202010110MIA 	 41.522
20 	 Deni Avdija 	 202101010MIN 	 41.285
21 	 LeBron James 	 202101010SAS 	 41.213
22 	 Tobias Harris 	 202012290PHI 	 40.844
23 	 Terrence Ross 	 202012270WAS 	 40.619
24 	 Mike Conley 	 202101010UTA

261 	 Serge Ibaka 	 202012290LAC 	 18.114
262 	 P.J. Washington 	 202012270CHO 	 18.068
263 	 Devin Vassell 	 202012230MEM 	 18.066
264 	 Doug McDermott 	 202012290IND 	 18.01
265 	 Deni Avdija 	 202012310WAS 	 17.869
266 	 Markelle Fultz 	 202012290OKC 	 17.815
267 	 Al Horford 	 202012280OKC 	 17.736
268 	 Daniel Theis 	 202012230BOS 	 17.694
269 	 Josh Richardson 	 202012270LAC 	 17.683
270 	 Jordan Clarkson 	 202012230POR 	 17.679
271 	 Thomas Bryant 	 202012310WAS 	 17.655
272 	 Aaron Holiday 	 202012270IND 	 17.655
273 	 James Johnson 	 202012270LAC 	 17.584
274 	 Domantas Sabonis 	 202012270IND 	 17.538
275 	 Brook Lopez 	 202012290MIA 	 17.505
276 	 Kyle Anderson 	 202012280BRK 	 17.5
277 	 Patty Mills 	 202012260SAS 	 17.491
278 	 Nickeil Alexander-Walker 	 202012310OKC 	 17.438
279 	 Clint Capela 	 202101010BRK 	 17.412
280 	 Monte Morris 	 202012290SAC 	 17.411
281 	 Andre Drummond 	 202012310IND 	 17.361
282 	 John Collins 	 202012230CHI 	 17.322
283 	 Devin Booker 	 202012

542 	 Tyrese Haliburton 	 202012260SAC 	 6.973
543 	 Jarrett Culver 	 202012230MIN 	 6.95
544 	 CJ Elleby 	 202012300LAC 	 6.936
545 	 Chuma Okeke 	 202012290OKC 	 6.935
546 	 Dejounte Murray 	 202012300SAS 	 6.916
547 	 Robin Lopez 	 202101010MIN 	 6.892
548 	 LaMelo Ball 	 202012230CLE 	 6.817
549 	 Kawhi Leonard 	 202012220LAL 	 6.81
550 	 Cody Martin 	 202012230CLE 	 6.809
551 	 Cameron Payne 	 202101010DEN 	 6.799
552 	 Justin Holiday 	 202012260CHI 	 6.794
553 	 Bobby Portis 	 202101010MIL 	 6.742
554 	 Harrison Barnes 	 202012230DEN 	 6.71
555 	 Jerami Grant 	 202101010DET 	 6.707
556 	 Jalen Brunson 	 202012270LAC 	 6.683
557 	 Cory Joseph 	 202012290SAC 	 6.673
558 	 Sindarius Thornwell 	 202012290PHO 	 6.548
559 	 Isaiah Hartenstein 	 202012250DEN 	 6.349
560 	 LaMelo Ball 	 202012300DAL 	 6.305
561 	 Javonte Green 	 202012290IND 	 6.291
562 	 Royce O'Neale 	 202012280OKC 	 6.207
563 	 Montrezl Harrell 	 202012270LAL 	 6.161
564 	 Nicolò Melli 	 202012290PHO 	 6.11
565 	 Bryn

761 	 Aleksej Pokusevski 	 202012290OKC 	 1.272
762 	 Georges Niang 	 202012280OKC 	 1.246
763 	 Semi Ojeleye 	 202012270IND 	 1.238
764 	 Justin James 	 202012270SAC 	 1.222
765 	 Brodric Thomas 	 202012260POR 	 1.209
766 	 Isaiah Hartenstein 	 202012280DEN 	 1.187
767 	 Georges Niang 	 202012260UTA 	 1.16
768 	 Kent Bazemore 	 202012250MIL 	 1.16
769 	 Tyrell Terry 	 202012250LAL 	 1.125
770 	 Danilo Gallinari 	 202012300BRK 	 1.124
771 	 Kyle Guy 	 202012270SAC 	 1.088
772 	 Moritz Wagner 	 202012230PHI 	 1.08
773 	 Hassan Whiteside 	 202012310HOU 	 1.068
774 	 T.J. Warren 	 202012270IND 	 1.056
775 	 Lamar Stevens 	 202012310IND 	 1.052
776 	 Tyler Johnson 	 202101010BRK 	 1.045
777 	 Miye Oni 	 202012280OKC 	 1.038
778 	 Tremont Waters 	 202012250BOS 	 1.037
779 	 Ryan Arcidiacono 	 202012260CHI 	 1.019
780 	 Adam Mokoka 	 202012260CHI 	 1.004
781 	 Georges Niang 	 202101010UTA 	 0.998
782 	 Wesley Iwundu 	 202012250LAL 	 0.996
783 	 Robin Lopez 	 202012310WAS 	 0.993
784 	 Bryn F

1047 	 Thon Maker 	 202012310IND 	 -2.223
1048 	 T.J. McConnell 	 202012290IND 	 -2.223
1049 	 Robert Williams 	 202101010DET 	 -2.251
1050 	 Jalen McDaniels 	 202012230CLE 	 -2.271
1051 	 Kelan Martin 	 202012260CHI 	 -2.273
1052 	 Precious Achiuwa 	 202012300MIA 	 -2.275
1053 	 Terrance Ferguson 	 202012310ORL 	 -2.276
1054 	 Keljin Blevins 	 202101010GSW 	 -2.29
1055 	 Tony Bradley 	 202012270CLE 	 -2.304
1056 	 Taurean Prince 	 202012220BRK 	 -2.319
1057 	 Donovan Mitchell 	 202012280OKC 	 -2.33
1058 	 Rudy Gay 	 202101010SAS 	 -2.344
1059 	 Nicolò Melli 	 202012250MIA 	 -2.356
1060 	 Jimmy Butler 	 202012230ORL 	 -2.389
1061 	 Jordan Poole 	 202012290DET 	 -2.396
1062 	 Kent Bazemore 	 202012220BRK 	 -2.399
1063 	 Jeff Green 	 202101010BRK 	 -2.405
1064 	 Kent Bazemore 	 202101010GSW 	 -2.412
1065 	 Talen Horton-Tucker 	 202101010SAS 	 -2.443
1066 	 Donte DiVincenzo 	 202012250MIL 	 -2.444
1067 	 Frank Ntilikina 	 202012230IND 	 -2.514
1068 	 Pat Connaughton 	 202012300MIA 	 -2.53

1332 	 Taurean Prince 	 202101010BRK 	 -8.228
1333 	 Vincent Poirier 	 202012310ORL 	 -8.267
1334 	 Bismack Biyombo 	 202012260CHO 	 -8.292
1335 	 Patrick Williams 	 202101010MIL 	 -8.33
1336 	 Marcus Smart 	 202012290IND 	 -8.441
1337 	 Grayson Allen 	 202012300BOS 	 -8.537
1338 	 Jeff Green 	 202012270CHO 	 -8.554
1339 	 Sviatoslav Mykhailiuk 	 202012260DET 	 -8.635
1340 	 Mikal Bridges 	 202101010DEN 	 -8.637
1341 	 Hassan Whiteside 	 202012270SAC 	 -8.665
1342 	 Patrick Patterson 	 202012270LAC 	 -8.666
1343 	 Cody Zeller 	 202012230CLE 	 -8.671
1344 	 Precious Achiuwa 	 202101010DAL 	 -8.721
1345 	 Nicolò Melli 	 202012230TOR 	 -8.726
1346 	 DeAndre Jordan 	 202101010BRK 	 -8.727
1347 	 Desmond Bane 	 202012300BOS 	 -8.765
1348 	 Nerlens Noel 	 202012290CLE 	 -8.796
1349 	 Denzel Valentine 	 202012260CHI 	 -8.825
1350 	 Bruno Caboclo 	 202012280DEN 	 -8.853
1351 	 Devonte' Graham 	 202012230CLE 	 -8.861
1352 	 Furkan Korkmaz 	 202012260NYK 	 -8.864
1353 	 Markieff Morris 	 2020122

1618 	 Domantas Sabonis 	 202012290IND 	 -21.839
1619 	 Tobias Harris 	 202012270CLE 	 -22.001
1620 	 Bojan Bogdanović 	 202101010UTA 	 -22.049
1621 	 James Wiseman 	 202101010GSW 	 -22.105
1622 	 Anthony Edwards 	 202012290LAC 	 -22.119
1623 	 Damian Lillard 	 202012300LAC 	 -22.143
1624 	 Josh Jackson 	 202012230MIN 	 -22.19
1625 	 Bam Adebayo 	 202012230ORL 	 -22.349
1626 	 OG Anunoby 	 202012230TOR 	 -22.462
1627 	 Reggie Bullock 	 202012230IND 	 -22.789
1628 	 Caris LeVert 	 202012270CHO 	 -22.9
1629 	 Deandre Ayton 	 202101010DEN 	 -23.013
1630 	 Brook Lopez 	 202012270NYK 	 -23.024
1631 	 Wendell Carter 	 202101010MIL 	 -23.099
1632 	 Bradley Beal 	 202012290WAS 	 -23.192
1633 	 Jerami Grant 	 202012230MIN 	 -23.367
1634 	 Thomas Bryant 	 202012290WAS 	 -23.478
1635 	 Ivica Zubac 	 202012270LAC 	 -23.595
1636 	 J.J. Redick 	 202012270NOP 	 -23.604
1637 	 Eric Paschall 	 202012250MIL 	 -23.678
1638 	 RJ Barrett 	 202012260NYK 	 -23.679
1639 	 Thaddeus Young 	 202101010MIL 	 -23.8

**SAVING AND LOADING MODELS

In [1212]:
###JUMP HERE FOR SAVING (SAVE)
torch.save(model2.state_dict(), "/Users/williamarnesen/Desktop/NBA_Model/model_embeddings.pt")
torch.save(model_rookies.state_dict(), "/Users/williamarnesen/Desktop/NBA_Model/model_rookies.pt")
torch.save(pc_model.state_dict(), "/Users/williamarnesen/Desktop/NBA_Model/model_matchups.pt")
torch.save(model_players.state_dict(), "/Users/williamarnesen/Desktop/NBA_Model/model_y2y.pt")


In [1531]:
with open("/Users/williamarnesen/Desktop/NBA_Model/embeddings.p","wb") as f:
    pickle.dump(embeddings, f)
    
model2 = model_v2(20)
model2.load_state_dict(torch.load("/Users/williamarnesen/Desktop/NBA_Model/model_embeddings.pt"))
model2.double()

model_rookies = model_rookie_projections()
model_rookies = model_rookies.double()
model_rookies.load_state_dict(torch.load("/Users/williamarnesen/Desktop/NBA_Model/model_rookies.pt"))


pc_model = PlayerCombinerModel()
pc_model.load_state_dict(torch.load("/Users/williamarnesen/Desktop/NBA_Model/model_matchups.pt"))

model_players = model_v1()
model_players = model_players.double()
model_players.load_state_dict(torch.load("/Users/williamarnesen/Desktop/NBA_Model/model_y2y.pt"))


<All keys matched successfully>

In [885]:
with open("/Users/williamarnesen/Desktop/NBA_Model/embeddings.p","wb") as f:
    pickle.dump(embeddings, f)
model2.load_state_dict


tensor(0.4693, grad_fn=<SelectBackward>)


In [893]:
for player in draft:
    year = draft[player][0]
    if year == 2020 and len(player) > 0:
        rookie_team = create_generic_team_with_player(player, 48, embeddings[player]['2020'])
        proj=pc_model([rookie_team],[base_list])
        print(player,round(proj[0,0].item(),3))

Anthony Edwards 0.469
James Wiseman 0.469
LaMelo Ball 0.469
Patrick Williams 0.469
Isaac Okoro 0.469
Onyeka Okongwu 0.469
Killian Hayes 0.469
Obi Toppin 0.469
Deni Avdija 0.469
Jalen Smith 0.469
Devin Vassell 0.469
Tyrese Haliburton 0.469
Kira Lewis 0.469
Aaron Nesmith 0.469
Cole Anthony 0.469
Isaiah Stewart 0.469
Aleksej Pokusevski 0.469
Josh Green 0.469
Saddiq Bey 0.469
Precious Achiuwa 0.469
Tyrese Maxey 0.469
Zeke Nnaji 0.469
Leandro Bolmaro 0.469
R.J. Hampton 0.469
Immanuel Quickley 0.469
Payton Pritchard 0.469
Udoka Azubuike 0.469
Jaden McDaniels 0.469
Malachi Flynn 0.469
Desmond Bane 0.469
Tyrell Terry 0.469
Vernon Carey 0.469
Daniel Oturu 0.469
Théo Maledon 0.469
Xavier Tillman 0.469
Tyler Bey 0.469
Vit Krejci 0.469
Saben Lee 0.469
Elijah Hughes 0.469
Robert Woodard 0.469
Tre Jones 0.469
Nick Richards 0.469
Jahmi'us Ramsey 0.469
Marko Simonovic 0.469
Jordan Nwora 0.469
CJ Elleby 0.469
Yam Madar 0.469
Nico Mannion 0.469
Isaiah Joe 0.469
Skylar Mays 0.469
Justinian Jessup 0.469
K